In [1]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

In [2]:
# Default datastore 
def_data_store = ws.get_default_datastore()

# Get the blob storage associated with the workspace
def_blob_store = Datastore(ws, "workspaceblobstore")

# Get file storage associated with the workspace
def_file_store = Datastore(ws, "workspacefilestore")

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = "cpu-cluster"
vm_size = "STANDARD_NC6"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,  # STANDARD_NC6 is GPU-enabled
                                                                min_nodes=0,
                                                                max_nodes=4)
    # create the compute target
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Found compute target: cpu-cluster


In [4]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
# `compute_target` as defined in "Azure Machine Learning compute" section above
aml_run_config.target = compute_target

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn','seaborn','tqdm'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]','seaborn','tqdm'], 
        pin_sdk_version=False)

In [5]:
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url and convert column "Survived" to boolean
web_path ='https://dprepdata.blob.core.windows.net/demo/Titanic.csv'
my_dataset = Dataset.Tabular.from_delimited_files(path=web_path, set_column_types={'Survived': DataType.to_bool()})

In [6]:
from azureml.pipeline.steps import PythonScriptStep
dataprep_source_dir = "./dataprep_src"
#entry_point = "prepare.py"
# `my_dataset` as defined above
ds_input = my_dataset.as_named_input('input1')

In [7]:
from azureml.data import OutputFileDatasetConfig
from azureml.core import Workspace, Datastore

datastore = ws.get_default_datastore()

output_data1 = OutputFileDatasetConfig(destination = (datastore, 'outputdataset/{run-id}'))
output_data_dataset = output_data1.register_on_complete(name = 'titanic_output_data')

In [8]:
train_source_dir = "./train_src"
train_entry_point = "train.py"

training_results = OutputFileDatasetConfig(name = "training_results",
    destination = def_blob_store)

    
train_step = PythonScriptStep(
    script_name=train_entry_point,
    source_directory=train_source_dir,
    arguments=["--input_data", ds_input],
    compute_target=compute_target, # , "--training_results", training_results
    runconfig=aml_run_config,
    allow_reuse=False
)

In [9]:
# list of steps to run (`compare_step` definition not shown)
compare_models = [train_step]

from azureml.pipeline.core import Pipeline

# Build the pipeline
pipeline1 = Pipeline(workspace=ws, steps=train_step)

In [10]:

pipeline1.validate()
print("Pipeline validation complete")

Step train.py is ready to be created [035d4106]
Pipeline validation complete


In [11]:
from azureml.core import Experiment

# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'Titanic_Pipeline_Notebook').submit(pipeline1)
pipeline_run1.wait_for_completion()

Created step train.py [035d4106][352f3455-009c-4f33-accf-790e24b3da21], (This step will run and generate new outputs)
Submitted PipelineRun 603c5ad4-2662-4223-bf96-d7ac3463622f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/603c5ad4-2662-4223-bf96-d7ac3463622f?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRunId: 603c5ad4-2662-4223-bf96-d7ac3463622f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/603c5ad4-2662-4223-bf96-d7ac3463622f?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 62a52900-ecba-459d-b197-ef140da349b1
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/62a52900-ecba-459d-b197-ef140da349b1?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/

'Finished'

In [12]:
from azureml.widgets import RunDetails

RunDetails(pipeline_run1).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
step_runs = pipeline_run1.get_children()
for step_run in step_runs:
    status = step_run.get_status()
    print('Script:', step_run.name, 'status:', status)
    
    # Change this if you want to see details even if the Step has succeeded.
    if status == "Failed":
        joblog = step_run.get_job_log()
        print('job log:', joblog)

Script: train.py status: Finished


In [14]:
from azureml.pipeline.core.graph import PipelineParameter

pipeline_param = PipelineParameter(
  name="pipeline_arg",
  default_value=10)

In [15]:
published_pipeline1 = pipeline_run1.publish_pipeline(
     name="Published_Titanic_Pipeline_Notebook",
     description="Titanic_Pipeline_Notebook Published Pipeline Description",
     version="1.0")

In [16]:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb

In [17]:
from azureml.pipeline.core import PublishedPipeline
import requests

response = requests.post(published_pipeline1.endpoint, 
                         json={"ExperimentName": "Titanic_Pipeline_Notebook",
                               "ParameterAssignments": {"pipeline_arg": 20}})

In [18]:
ws

Workspace.create(name='mlopsdev', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops')

Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


In [22]:
from azureml.pipeline.core import PipelineEndpoint

published_pipeline = PipelineEndpoint.get(workspace=ws, name="Published_Titanic_Pipeline_Notebook")
pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="TitanicPipelineEndpointTest",
                                            pipeline=published_pipeline, description="Test Published_Titanic_Pipeline_Notebook description Notebook")

ErrorResponseException: (UserError) Response status code does not indicate success: 400 (Bad Request).
Microsoft.RelInfra.Common.Exceptions.ErrorResponseException: PipelineEndpoint name Published_Titanic_Pipeline_Notebook not found in workspace 0fb30f4c-f96f-4ccb-98e8-b741ce9e8f19

Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https:/

In [ ]:
pipeline_endpoint_by_name = PipelineEndpoint.get(workspace=ws, name="Published_Titanic_Pipeline_Notebook")
run_id = pipeline_endpoint_by_name.submit("PipelineEndpointExperiment")
print(run_id)

Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
Class SynapseCompute: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
